In [1]:
# Simple thermostat API using FastAPI
# importing libraries
from fastapi import FastAPI
import uvicorn
import nest_asyncio
import random
import json

app = FastAPI()


In [ ]:
# defining some variables
global DEVICE_ID
DEVICE_ID = "device_1"
